In [64]:
import pandas as pd
import requests
import csv
from bs4 import BeautifulSoup

In [93]:
Member_URLS = []

base_url = 'https://www.gabar.org/membersearchresults.cfm?start={}&id={}'
pages = [1, 26, 51]
id = 'A06532F0-E305-D491-EFFB8852D50B8F99'

for page in pages:
    url = base_url.format(page, id)
    response = requests.get(url)

    soup = BeautifulSoup(response.content, 'html.parser')
    member_names = soup.find_all('div', {'class': 'member-name'})

    for member_name in member_names:
        links = member_name.find_all('a')
        for link in links:
            print(link.get('href'))
            Member_URLS.append(f"https://www.gabar.org"+str(link.get('href')))
print(Member_URLS)

[]


In [98]:
#get urls of member profiles
Member_URLS = []


for page_num in range(1, 2):
    url = f'https://www.gabar.org/membersearchresults.cfm?page={page_num}'
    response = requests.get(url)

    soup = BeautifulSoup(response.content, 'html.parser')
    member_names = soup.find_all('div', {'class': 'member-name'})

    for member_name in member_names:
        links = member_name.find_all('a')
        for link in links:
            Member_URLS.append(f"https://www.gabar.org"+str(link.get('href')))

print(Member_URLS)

['https://www.gabar.org/MemberSearchDetail.cfm?ID=Njc0MTg3', 'https://www.gabar.org/MemberSearchDetail.cfm?ID=MDAwMDMx', 'https://www.gabar.org/MemberSearchDetail.cfm?ID=ODkwMTE2', 'https://www.gabar.org/MemberSearchDetail.cfm?ID=MDAwMDM1', 'https://www.gabar.org/MemberSearchDetail.cfm?ID=MDAwMDUw', 'https://www.gabar.org/MemberSearchDetail.cfm?ID=MzMzOTI0', 'https://www.gabar.org/MemberSearchDetail.cfm?ID=MDAwMDU1', 'https://www.gabar.org/MemberSearchDetail.cfm?ID=MDAwMDc1', 'https://www.gabar.org/MemberSearchDetail.cfm?ID=MDAwMDc5', 'https://www.gabar.org/MemberSearchDetail.cfm?ID=MDAwMDg3', 'https://www.gabar.org/MemberSearchDetail.cfm?ID=MTQyMjEw', 'https://www.gabar.org/MemberSearchDetail.cfm?ID=MjYyMjEx', 'https://www.gabar.org/MemberSearchDetail.cfm?ID=MDc2MTEx', 'https://www.gabar.org/MemberSearchDetail.cfm?ID=MDAwMDg5', 'https://www.gabar.org/MemberSearchDetail.cfm?ID=MjExNTg5', 'https://www.gabar.org/MemberSearchDetail.cfm?ID=MzE1MTYx', 'https://www.gabar.org/MemberSearchDeta

In [65]:
#get urls of member profiles
Member_URLS = []


for page_num in range(1, 2):
    url = f'https://www.gabar.org/membersearchresults.cfm?page={page_num}'
    response = requests.get(url)

    soup = BeautifulSoup(response.content, 'html.parser')
    member_names = soup.find_all('div', {'class': 'member-name'})

    for member_name in member_names:
        links = member_name.find_all('a')
        for link in links:
            Member_URLS.append(f"https://www.gabar.org"+str(link.get('href')))

#use the mmber profile links to scrape the contact infor for each member
contact_info = []

for url in Member_URLS:
    # Send a GET request to the URL and retrieve the HTML content
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    try:
        # Find all the divs with class 'course-box'
        course_boxes = soup.find_all('div', {'class': 'course-box'})
        course_ids = soup.find('div', {'class': 'course-id'})

        name = (course_ids.get_text())
    except:
        None

    # Loop through each course box and extract the relevant information
    email_list = []
    try:
        course_boxes = soup.find_all('div', {'class': 'course-box'})

        for box in course_boxes:
            td_tags = box.find_all('td')
            for td in td_tags:
                email_link = td.find('a', href=True)
                if email_link and email_link['href'].startswith('mailto:'):
                    email_list.append(email_link['href'][7:])

        email = (email_list[0])

        # Create a dictionary to store the course information
        course = {
            'name': name,
            'email': email,
        }

        # Add the course information to the list
        contact_info.append(course)

    except:
        None

# Open a new file in "write" mode and specify the delimiter as a comma
with open('contact_info.csv', mode='w', newline='') as csv_file:
    writer = csv.writer(csv_file, delimiter=',')

    # Write the header row
    writer.writerow(['Name', 'Email'])

    # Iterate over each dictionary in the list and write a new row for each
    for item in contact_info:
        writer.writerow([item['name'], item['email']])

# Convert the list of courses to a CSV
# df = pd.DataFrame(contact_info)

# # Print the DataFrame
# print(contact_info)
# display(df)

# df.to_csv('contact_info.csv', index=False)

In [112]:
import requests
from bs4 import BeautifulSoup

# create an empty list to store the LegalProfile URLs
legal_profile_urls = []

# specify the range of pages to scrape
start_page = 0
end_page = 1

# loop through the pages in the specified range
for page_number in range(start_page, end_page+1):
    # make a request to the current page
    url = f'https://www.mywsba.org/PersonifyEbusiness/LegalDirectory.aspx?ShowSearchResults=TRUE&Country=USA&Page={page_number}'
    response = requests.get(url)
    
    # create a BeautifulSoup object from the response content
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # find all the <tr> elements with class "grid-row"
    rows = soup.find_all('tr', {'class': 'grid-row'})
    
    # loop through the rows and extract the LegalProfile URL from the onclick attribute
    for row in rows:
        onclick = row.get('onclick')
        if onclick:
            url = onclick.split("'")[1]
            legal_profile_urls.append(f"https://www.mywsba.org/PersonifyEbusiness/"+str(url))

# print the list of LegalProfile URLs
len(legal_profile_urls)

['https://www.mywsba.org/PersonifyEbusiness/LegalDirectory/LegalProfile.aspx?Usr_ID=000000000001', 'https://www.mywsba.org/PersonifyEbusiness/LegalDirectory/LegalProfile.aspx?Usr_ID=000000000497', 'https://www.mywsba.org/PersonifyEbusiness/LegalDirectory/LegalProfile.aspx?Usr_ID=000000000002', 'https://www.mywsba.org/PersonifyEbusiness/LegalDirectory/LegalProfile.aspx?Usr_ID=000009101360', 'https://www.mywsba.org/PersonifyEbusiness/LegalDirectory/LegalProfile.aspx?Usr_ID=000000000003', 'https://www.mywsba.org/PersonifyEbusiness/LegalDirectory/LegalProfile.aspx?Usr_ID=000009101361', 'https://www.mywsba.org/PersonifyEbusiness/LegalDirectory/LegalProfile.aspx?Usr_ID=000000000004', 'https://www.mywsba.org/PersonifyEbusiness/LegalDirectory/LegalProfile.aspx?Usr_ID=000009101362', 'https://www.mywsba.org/PersonifyEbusiness/LegalDirectory/LegalProfile.aspx?Usr_ID=000000000005', 'https://www.mywsba.org/PersonifyEbusiness/LegalDirectory/LegalProfile.aspx?Usr_ID=000009062720', 'https://www.mywsba

40

In [138]:
import requests
from bs4 import BeautifulSoup
import csv


# create an empty list to store the LegalProfile URLs
legal_profile_urls = []

# specify the range of pages to scrape there are 1734 pages
start_page = 901
end_page = 1100

# loop through the pages in the specified range
for page_number in range(start_page, end_page+1):
    # make a request to the current page
    url = f'https://www.mywsba.org/PersonifyEbusiness/LegalDirectory.aspx?ShowSearchResults=TRUE&EligibleToPractice=Y&Country=USA&Page={page_number}'
    response = requests.get(url)
    
    # create a BeautifulSoup object from the response content
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # find all the <tr> elements with class "grid-row"
    rows = soup.find_all('tr', {'class': 'grid-row'})
    
    # loop through the rows and extract the LegalProfile URL from the onclick attribute
    for row in rows:
        onclick = row.get('onclick')
        if onclick:
            url = onclick.split("'")[1]
            legal_profile_urls.append(f"https://www.mywsba.org/PersonifyEbusiness/"+str(url))

    print(page_number)

# initialize empty list to store scraped data
data = []

# loop through each URL and scrape the data
for url in legal_profile_urls:
    # make a GET request to the URL
    response = requests.get(url)


    # parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # find the relevant elements by ID and extract the text
    member_no = soup.find('span', {'id': 'dnn_ctr2977_DNNWebControlContainer_ctl00_lblMemberNo'}).text
    license_type = soup.find('span', {'id': 'dnn_ctr2977_DNNWebControlContainer_ctl00_lblLicenseType'}).text
    eligible_to_practice = soup.find('span', {'id': 'dnn_ctr2977_DNNWebControlContainer_ctl00_lblEligibleToPractice'}).text
    status = soup.find('span', {'id': 'dnn_ctr2977_DNNWebControlContainer_ctl00_lblStatus'}).text
    wa_admit_date = soup.find('span', {'id': 'dnn_ctr2977_DNNWebControlContainer_ctl00_lblWaAdmitDate'}).text
    email = soup.find('span', {'id': 'dnn_ctr2977_DNNWebControlContainer_ctl00_lblEmail'}).text
    member_name = soup.find('span', {'id': 'dnn_ctr2977_DNNWebControlContainer_ctl00_lblMemberName'}).text
    address = soup.find('span', {'id': 'dnn_ctr2977_DNNWebControlContainer_ctl00_lblAddress'}).text
    phone = soup.find('span', {'id': 'dnn_ctr2977_DNNWebControlContainer_ctl00_lblPhone'}).text
    area_of_practice = soup.find('span', {'id': 'dnn_ctr2977_DNNWebControlContainer_ctl00_lblPracticeAreas'}).text

    # append the data to the list
    data.append([member_name, member_no, license_type, eligible_to_practice, status, wa_admit_date, email, address, phone, area_of_practice])
    print(url)
    
# write the data to a CSV file
#with open('Washington_Bar_Association.csv', 'w', newline='') as file:
with open('Washington_Bar_Association.csv', 'a', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Full_Name','Member_No.', 'License_Type', 'Eligible_to_Practice', 'Status', 'WSBA_Admit_Date', 'Email','address','phone','legal_speciality'])
    writer.writerows(data)

901
902
903
904
905
906
907
908
909
910
911
912
913
914
915
916
917
918
919
920
921
922
923
924
925
926
927
928
929
930
931
932
933
934
935
936
937
938
939
940
941
942
943
944
945
946
947
948
949
950
951
952
953
954
955
956
957
958
959
960
961
962
963
964
965
966
967
968
969
970
971
972
973
974
975
976
977
978
979
980
981
982
983
984
985
986
987
988
989
990
991
992
993
994
995
996
997
998
999
1000
1001
1002
1003
1004
1005
1006
1007
1008
1009
1010
1011
1012
1013
1014
1015
1016
1017
1018
1019
1020
1021
1022
1023
1024
1025
1026
1027
1028
1029
1030
1031
1032
1033
1034
1035
1036
1037
1038
1039
1040
1041
1042
1043
1044
1045
1046
1047
1048
1049
1050
1051
1052
1053
1054
1055
1056
1057
1058
1059
1060
1061
1062
1063
1064
1065
1066
1067
1068
1069
1070
1071
1072
1073
1074
1075
1076
1077
1078
1079
1080
1081
1082
1083
1084
1085
1086
1087
1088
1089
1090
1091
1092
1093
1094
1095
1096
1097
1098
1099
1100
https://www.mywsba.org/PersonifyEbusiness/LegalDirectory/LegalProfile.aspx?Usr_ID=000000040375
http